In [1]:
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
import pandas as pd
from pyspark.sql.functions import udf
from pandas import pivot_table
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType

In [2]:
spark_df = sqlContext.read.format('csv').options(header='true', inferSchema = True).load('/FileStore/tables/ecommerce.csv')
#spark_df = spark.read.csv('/FileStore/tables/', header=True, schema=sc);

data = pd.read_csv('/dbfs/FileStore/tables/ecommerce.csv', encoding = "ISO-8859-1")

#data = spark.read.csv('/FileStore/tables/newdata.csv', header=True, schema=sc)

In [3]:
spark_df.printSchema()

In [4]:
def get_hash(x):
  return abs(hash(x)) % 10**9

spark_df = udf(lambda z: get_hash(z), FloatType())
spark_df['id'] = spark_df['id'].apply(get_hash)

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

cols = data.columns
stages = []
# Assemble our attributes into one column using Vector Assembler
numericCols = ["id","username"]
labelString = StringIndexer(inputCol = "rating", outputCol = "label").setHandleInvalid("keep")
stages += [labelString]


assemblercols = ['rating']
assembler = VectorAssembler(inputCols=assemblercols, outputCol="features")
stages += [assembler]

# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(data)

data = pipelineModel.transform(data)
# Keep relevant columns
selectedcols = ["label", "features"] + cols
data = data.select(selectedcols) 


display(data)